In [3]:
import os
from os.path import join as opj
import numpy as np
from torch.utils.data import Dataset
import h5py
import json
import numpy as np
from transforms3d.quaternions import quat2mat
from transforms3d.euler import euler2mat
import pickle as pkl

In [9]:
dataset=  AffordNetDataset(data_dir = "/data1/linfeng/affordancenet/",split="train" )

In [14]:
data,data,label,idx,clas=dataset[200]

In [16]:
label.shape

(2048, 18)

In [4]:
def random_rotation_matrix():
    rand = np.random.rand(3)
    r1 = np.sqrt(1.0 - rand[0])
    r2 = np.sqrt(rand[0])
    pi2 = np.pi * 2.0
    t1 = pi2 * rand[1]
    t2 = pi2 * rand[2]
    q = np.array([np.cos(t2)*r2, np.sin(t1)*r1, np.cos(t1)*r1, np.sin(t2)*r2])
    return quat2mat(q)


def rotate_point_cloud_SO3(batch_data):
    """
    Randomly rotate the point clouds to augument the dataset
    rotation is per shape based along three axis
    Input:
        BxNx3 array, original batch of point clouds
    Return:
        BxNx3 array, rotated batch of point clouds
    """
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_matrix = random_rotation_matrix()
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = (
            np.matmul(rotation_matrix, shape_pc.reshape((-1, 3)).T)).T
    return rotated_data


def rotate_point_cloud_y(batch_data):
    """ Randomly rotate the point clouds to augument the dataset
        rotation is per shape based along up direction
        Input:
          BxNx3 array, original batch of point clouds
        Return:
          BxNx3 array, rotated batch of point clouds
    """
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_angle = np.random.uniform() * 2 * np.pi
        rotation_matrix = euler2mat(0, rotation_angle, 0)
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = (
            np.matmul(rotation_matrix, shape_pc.reshape((-1, 3)).T)).T
    return rotated_data

In [8]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc, centroid, m

class AffordNetDataset(Dataset):
    def __init__(self, data_dir, split, partial=False, rotate='None', semi=False):
        super().__init__()
        self.data_dir = data_dir
        self.split = split

        self.partial = partial
        self.rotate = rotate
        self.semi = semi

        self.load_data()

        self.affordance = self.all_data[0]["affordance"]

        return

    def load_data(self):
        self.all_data = []
        if self.semi:
            with open(opj(self.data_dir, 'semi_label_1.pkl'), 'rb') as f:
                temp_data = pkl.load(f)
        else:
            if self.partial:
                with open(opj(self.data_dir, 'partial_%s_data.pkl' % self.split), 'rb') as f:
                    temp_data = pkl.load(f)
            elif self.rotate != "None" and self.split != 'train':
                with open(opj(self.data_dir, 'rotate_%s_data.pkl' % self.split), 'rb') as f:
                    temp_data_rotate = pkl.load(f)
                with open(opj(self.data_dir, 'full_shape_%s_data.pkl' % self.split), 'rb') as f:
                    temp_data = pkl.load(f)
            else:
                with open(opj(self.data_dir, 'full_shape_%s_data.pkl' % self.split), 'rb') as f:
                    temp_data = pkl.load(f)
        for index, info in enumerate(temp_data):
            if self.partial:
                partial_info = info["partial"]
                for view, data_info in partial_info.items():
                    temp_info = {}
                    temp_info["shape_id"] = info["shape_id"]
                    temp_info["semantic class"] = info["semantic class"]
                    temp_info["affordance"] = info["affordance"]
                    temp_info["view_id"] = view
                    temp_info["data_info"] = data_info
                    self.all_data.append(temp_info)
            elif self.split != 'train' and self.rotate != 'None':
                rotate_info = temp_data_rotate[index]["rotate"][self.rotate]
                full_shape_info = info["full_shape"]
                for r, r_data in rotate_info.items():
                    temp_info = {}
                    temp_info["shape_id"] = info["shape_id"]
                    temp_info["semantic class"] = info["semantic class"]
                    temp_info["affordance"] = info["affordance"]
                    temp_info["data_info"] = full_shape_info
                    temp_info["rotate_matrix"] = r_data.astype(np.float32)
                    self.all_data.append(temp_info)
            else:
                temp_info = {}
                temp_info["shape_id"] = info["shape_id"]
                temp_info["semantic class"] = info["semantic class"]
                temp_info["affordance"] = info["affordance"]
                temp_info["data_info"] = info["full_shape"]
                self.all_data.append(temp_info)

    def __getitem__(self, index):

        data_dict = self.all_data[index]
        modelid = data_dict["shape_id"]
        modelcat = data_dict["semantic class"]

        data_info = data_dict["data_info"]
        model_data = data_info["coordinate"].astype(np.float32)
        labels = data_info["label"]
        for aff in self.affordance:
            temp = labels[aff].astype(np.float32).reshape(-1, 1)
            model_data = np.concatenate((model_data, temp), axis=1)

        datas = model_data[:, :3]
        targets = model_data[:, 3:]

        if self.rotate != 'None':
            if self.split == 'train':
                if self.rotate == 'so3':
                    datas = rotate_point_cloud_SO3(
                        datas[np.newaxis, :, :]).squeeze()
                elif self.rotate == 'z':
                    datas = rotate_point_cloud_y(
                        datas[np.newaxis, :, :]).squeeze()
            else:
                r_matrix = data_dict["rotate_matrix"]
                datas = (np.matmul(r_matrix, datas.T)).T

        datas, _, _ = pc_normalize(datas)

        return datas, datas, targets, modelid, modelcat

    def __len__(self):
        return len(self.all_data)

In [17]:
with open(opj("/data1/linfeng/affordancenet", 'full_shape_%s_data.pkl' % "train"), 'rb') as f:
                    temp_data = pkl.load(f)

In [31]:
temp_data[0]

{'shape_id': '63e0b01c60c1a0edfafd17eed9590afe',
 'semantic class': 'Door',
 'affordance': ['grasp',
  'contain',
  'lift',
  'openable',
  'layable',
  'sittable',
  'support',
  'wrap_grasp',
  'pourable',
  'move',
  'displaY',
  'pushable',
  'pull',
  'listen',
  'wear',
  'press',
  'cut',
  'stab'],
 'full_shape': {'coordinate': array([[ 0.15929998, -0.7428185 , -0.00485236],
         [-0.22462437,  0.972573  ,  0.00416868],
         [ 0.221295  ,  0.1727677 ,  0.00374921],
         ...,
         [-0.10196283,  0.636612  , -0.01006717],
         [ 0.0958824 , -0.9201699 ,  0.00974288],
         [-0.16395783,  0.60726637,  0.00812608]], dtype=float32),
  'label': {'grasp': array([[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]], dtype=float32),
   'contain': array([[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]], dtype=float32),
   'lift': array([[0.],
          [0.],
        